In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def f_func(z):
    return norm.pdf(z) + z*norm.cdf(z)

# Here we have to remember that x indexing goes from 1 to M
def sigma(S, x, lambda_):
    M = S.shape[0]
    e_x = np.zeros([M,1])
    e_x[x-1] = 1
    
    return np.dot(S, e_x)/np.sqrt(lambda_[x-1]+S[x-1][x-1])

### Algorithm 1

In [3]:
def algorithm_1 (a, b):
    M = len(a)
    c = []
    A = []
    
    # Initialization
    c.append(-np.inf)
    c.append(np.inf)
    A.append(1)
    loop_done=False
    
    # Main loop
    for i in range(1, M):
        c.append(np.inf)

        while loop_done == False:
            j = A[len(A)-1]

            c[j] = (a[j-1]-a[i+1-1])/(b[i+1-1]-b[j-1])

            if len(A)!=1 and c[j]<=c[A[len(A)-1-1]]:
                A.pop()
                loop_done = False
            else:
                loop_done = True
        A.append(i+1)
        loop_done = False
    
    return c, A

### KG-Algorithm

In [4]:
def KG_Alg(mu, S, lambda_):
    M = S.shape[0]
    xx = -1
    vv = -np.inf
    L=[]
    for x in range(1,M):
        a = list(mu)
        b = list(sigma(S, x, lambda_)[:,0])


        # Lexicographically sort a and b (with b first)
        data = list(zip(a,b))
        data.sort(key=lambda pair : (pair[1], pair[0]))
        a, b = (list(t) for t in zip(*data))

        for i in range(1, M-1):
            if b[i-1]==b[i+1-1]:
                # Remove entry i-1 from data pairs sequences
                del(data[i-1])
                del(b[i-1])
                del(a[i-1])
        # Use Algorithm 1 to obtain c and A 
        c, A = algorithm_1 (a, b)

        # Obtain restricions of a, b, c to 
        # indexes from A (we have to adjust
        # due to the index disparity of c, a
        # starting from 1 but a, b start from 0)
        # From now on a, b, c, A have the same index set!!!
        a = [a[j-1] for j in A]
        b = [b[j-1] for j in A]
        c = [c[j] for j in A]
        M = len(A)

        v = np.log(np.sum([(b[j+1]-b[j])*f_func(-np.abs(c[j])) for j in range(0,M-1)]))

        #Debug
        print("{}: {}".format(x,v))
        L.append(v)

        if x==1 or v>vv:
            vv = v
            xx = x
    return xx-1, vv

In [5]:
def KG_single(mu, S, lambda_, x):
    M = S.shape[0]
    a = list(mu)
    b = list(sigma(S, x, lambda_)[:,0])

    # Lexicographically sort a and b (with b first)
    data = list(zip(a,b))
    data.sort(key=lambda pair : (pair[1], pair[0]))
    a, b = (list(t) for t in zip(*data))

    for i in range(1, M-1):
        if b[i-1]==b[i+1-1]:
            # Remove entry i-1 from data
            #pairs sequences
            del(data[i-1])
            del(b[i-1])
            del(a[i-1])
    # Use Algorithm 1 to obtain c and A 
    c, A = algorithm_1(a, b)

    # Obtain restricions of a, b, c to 
    # indexes from A (we have to adjust
    # due to the index disparity of c, a
    # starting from 1 but a, b start from 0)
    # From now on a, b, c, A have the same index set!!!
    a = [a[j-1] for j in A]
    b = [b[j-1] for j in A]
    c = [c[j] for j in A]
    M = len(A)

    v = np.log(np.sum([(b[j+1]-b[j])*f_func(-np.abs(c[j])) 
                      for j in range(0,M-1)]))

    return x-1, v

In [6]:
def KG_Alg2(mu, S, lambda_):
    M = S.shape[0]
    L=[]
    
    for x in range(1,M):
        L.append(KG_single(mu, S, lambda_, x))
        
    L.sort(key=lambda pair : pair[1], reverse=True)
    return L[0]

### Test Runs

In [7]:
S = np.array([[0,  1,  2,  3,  4],
              [5,  6,  7,  8,  9],
              [10, 11, 12, 13, 14],
              [15, 16, 17, 18, 19],
              [11, 12, 13, 14, 15]])
lambda_ = np.array([0.2, 0.1, 0.3, 0.12, 0.4])

# a and b are vectors of dim M, and b has strictly increasing values
M = S.shape[0] #M=5 in our example
a=[0.12, 0.10, 0.11, 1.20, 0.0]
b=[0.1,  0.31, 0.33, 0.42, 0.45]

# M=7
# a=list(np.random.uniform(size=M))
# b=sorted(list(np.random.uniform(size=M)))

In [8]:
c,A = algorithm_1(a,b)
print(c)
print(A)

[-inf, -3.375000000000001, -0.49999999999999933, -12.111111111111114, 39.999999999999964, inf]
[1, 4, 5]


In [9]:
from nearest_correlation import nearcorr
from scipy.stats import random_correlation

np.random.seed(126)

# G = np.array([[0.20,  0.10,  0.10,  0.15, 0.22],
#               [0.10,  0.30, -0.11, -0.16, 0.12],
#               [0.10, -0.11,  0.10, -0.17, 0.63],
#               [0.15, -0.16, -0.17,  0.18, 0.51],
#               [0.22,  0.12,  0.63,  0.51, 0.40]])

g=7/sum([.5, .8, 1.2, 2.5, 1.7, 2.1, 2.2])
G = np.round(random_correlation.rvs((g*.5, g*.8, g*1.2, 
                                     g*2.5, g*1.7, g*2.1, 
                                     g*2.2)), 3)


S = nearcorr(G, tol=[], flag=0, max_iterations=1000,
               n_pos_eig=0, weights=None, verbose=False,
               except_on_too_many_iterations=True)
M = S.shape[0]
lambda_ = np.array([0.2, 1.1, 1.3, 0.12, 0.4, 0.3, 0.12])
mu = np.array([0.2, 0.21, 0.92, 0.11, 0.7, 0.2, -0.1])

In [10]:
KG_Alg(mu, S, lambda_)

1: -2.2526178371099967
2: -3.545497654948802
3: -1.887239780817405
4: -1.9574218935030208
5: -1.5260580309321654
6: -1.9096914131233804


(4, -1.5260580309321654)

In [11]:
KG_Alg2(mu, S, lambda_)

(4, -1.5260580309321654)

## Update Functions

##### Data:

In [12]:
from nearest_correlation import nearcorr
from scipy.stats import random_correlation

np.random.seed(126)

g =7/sum([.5, .8, 1.2, 2.5, 1.7, 2.1, 2.2])
G = np.round(random_correlation.rvs((g*.5, g*.8, g*1.2, 
                                     g*2.5, g*1.7, g*2.1, 
                                     g*2.2)), 3)
S = nearcorr(G, tol=[], flag=0, max_iterations=1000,
               n_pos_eig=0, weights=None, verbose=False,
               except_on_too_many_iterations=True)

lambda_ = np.array([0.2, 1.1, 1.3, 0.12, 0.4, 0.3, 0.12])

mu = np.array([0.2, 0.21, 0.92, 0.11, 0.7, 0.2, -0.1])

y = 0.14

#x indexing goes from 1 to M
x = 4

In [13]:
def update_mu_S(mu, S, lambda_, x, y):
    # Adjust indexing to python lists
    x = x-1
    
    # Get e_x
    M = S.shape[0]
    e_x = np.zeros([M,1])
    e_x[x] = 1
    
    mu_1 = mu.reshape(M,1) + ((y-mu[x])/(lambda_[x]+S[x][x])*np.dot(S, e_x))
    
    S_1 = S - (1/(lambda_[x]*S[x][x]))*S.dot(e_x).dot(e_x.transpose()).dot(S)
    
    return mu_1, S_1

In [14]:
import Levenshtein

Levenshtein.distance('110','1111')

2

In [15]:
import textdistance

In [16]:
textdistance.levenshtein('0111','1111')

1

### Exponential Quadratic Kernel

In [131]:
#Given data
partition_list=[]

#Elements of partition list
p1={"constraints":{0:0, 1:0}}
p2={"constraints":{0:0, 1:0, 2:0, 2:1}}
p3={"constraints":{0:1, 1:0}}

partition_list = [p1, p2, p3]

# Tuning parameters
s = 0.2
l = 1.0

In [129]:
m = len(partition_list)
S = np.empty([m, m])


In [ ]:
''.join(map(str, list(partition_list[2]["constraints"].values())))

''.join([str(elem) for elem in constraints])

In [143]:
x = np.array([[('Rex', 9, 81.0), ('Fido', 3, 27.0)],
             [('lal', 10, 71.0), ('iuaa', 3, 21.1)]],
             dtype=[('name', 'U10'), ('age', 'i4'), ('weight', 'f4')])

In [145]:
x['age']

array([[ 9,  3],
       [10,  3]])

In [142]:
print(x)

[[('Rex', 9, 81.) ('Fido', 3, 27.)]]
